# 03 - Ball & Player Detection

Module 2: Comparing Feature Engineering vs Feature Learning approaches.

Methods compared:
1. **Classical (Feature Engineering)**: Viola-Jones + HOG + SVM, Color Segmentation + Hough Circle
2. **YOLOv8 (Feature Learning)**: Single-stage detector fine-tuned on tennis/pickleball
3. **TrackNet (Feature Learning)**: Specialized heatmap network for small ball detection

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import sys, os
sys.path.insert(0, os.path.abspath('..'))

from src.object_detection import (
    ClassicalPlayerDetector,
    ClassicalBallDetector,
    YOLODetector,
    TrackNetDetector,
    FasterRCNNDetector,
    TrackNet,
    FocalLoss,
    non_max_suppression,
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Classical Detection (Feature Engineering Baseline)

In [ ]:
# Load sample frame
VIDEO_PATH = '../data/raw/sample_match.mp4'

if os.path.exists(VIDEO_PATH):
    cap = cv2.VideoCapture(VIDEO_PATH)
    cap.set(cv2.CAP_PROP_POS_FRAMES, 100)
    _, frame = cap.read()
    cap.release()
else:
    frame = np.random.randint(50, 200, (720, 1280, 3), dtype=np.uint8)
    print('Using placeholder frame - load a real video for meaningful results')

# Classical Player Detection: HOG + SVM
hog_detector = ClassicalPlayerDetector(method='hog')
start = time.time()
hog_detections = hog_detector.detect(frame)
hog_time = time.time() - start

print(f'HOG+SVM Player Detection:')
print(f'  Detections: {len(hog_detections)}')
print(f'  Time: {hog_time*1000:.1f} ms')

# Classical Ball Detection: Color + Hough Circle
ball_detector = ClassicalBallDetector()
start = time.time()
ball_detections = ball_detector.detect(frame)
ball_time = time.time() - start

print(f'\nColor+Hough Ball Detection:')
print(f'  Detections: {len(ball_detections)}')
print(f'  Time: {ball_time*1000:.1f} ms')

In [ ]:
# Visualize classical detections
vis = frame.copy()
for det in hog_detections:
    x1, y1, x2, y2 = det.bbox
    cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(vis, f'Player {det.confidence:.2f}', (x1, y1-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

for det in ball_detections:
    cx, cy = int(det.center[0]), int(det.center[1])
    cv2.circle(vis, (cx, cy), 8, (0, 255, 255), 2)
    cv2.putText(vis, 'Ball', (cx+10, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
plt.title('Classical Detection: HOG Players (green) + Hough Ball (yellow)')
plt.axis('off')
plt.show()

## 2. YOLOv8 Detection (Feature Learning)

In [ ]:
# YOLOv8 Detection
# Uses pretrained COCO model (can detect 'person' and 'sports ball')
# For best results, fine-tune on tennis/pickleball dataset

try:
    yolo = YOLODetector(
        model_path='yolov8s.pt',  # Will auto-download
        conf_threshold=0.25,
        classes=[0, 32],  # COCO: 0=person, 32=sports ball
    )
    
    start = time.time()
    yolo_detections = yolo.detect(frame)
    yolo_time = time.time() - start
    
    print(f'YOLOv8 Detection:')
    print(f'  Detections: {len(yolo_detections)}')
    print(f'  Time: {yolo_time*1000:.1f} ms')
    for det in yolo_detections:
        print(f'    {det}')

except Exception as e:
    print(f'YOLOv8 not available: {e}')
    print('Install: pip install ultralytics')

## 3. TrackNet Architecture (Specialized Ball Detection)

In [ ]:
import torch

# TrackNet model architecture
model = TrackNet(input_frames=3, output_channels=1)
print('TrackNet Architecture:')
print(f'  Input: 3 consecutive frames (9 channels, 360x640)')
print(f'  Output: 1-channel heatmap (360x640)')
print(f'  Parameters: {sum(p.numel() for p in model.parameters()):,}')
print()

# Test forward pass
dummy_input = torch.randn(1, 9, 360, 640)
with torch.no_grad():
    output = model(dummy_input)
print(f'  Input shape: {dummy_input.shape}')
print(f'  Output shape: {output.shape}')
print(f'  Output range: [{output.min():.3f}, {output.max():.3f}]')

In [ ]:
# Visualize TrackNet heatmap generation (ground truth)
from src.object_detection import TrackNetDetector

# Generate sample heatmap for ball at position (320, 180)
heatmap = TrackNetDetector.generate_heatmap(320, 180, 640, 360, sigma=2.5)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].imshow(heatmap, cmap='hot')
axes[0].set_title('Ground Truth Heatmap (ball at 320, 180)')
axes[0].plot(320, 180, 'g+', markersize=15, markeredgewidth=2)

axes[1].imshow(heatmap[170:190, 310:330], cmap='hot', interpolation='nearest')
axes[1].set_title('Zoomed Gaussian Peak')

plt.tight_layout()
plt.show()

## 4. Feature Engineering vs Feature Learning Comparison

In [ ]:
# Comparison summary
comparison = {
    'Method': ['Viola-Jones+HOG', 'YOLOv8s', 'TrackNet', 'Faster R-CNN'],
    'Type': ['Feature Engineering', 'Feature Learning', 'Feature Learning', 'Feature Learning'],
    'Target': ['Players', 'Ball+Players', 'Ball', 'Players'],
    'Approach': ['Hand-crafted features', 'Single-stage CNN', '3-frame heatmap CNN', 'Two-stage CNN'],
    'Speed': ['Fast (~30fps)', 'Fast (~45fps)', 'Medium (~25fps)', 'Slow (~15fps)'],
    'Small Objects': ['Poor', 'Moderate', 'Excellent', 'Moderate'],
}

print('=' * 90)
print('Feature Engineering vs Feature Learning Comparison')
print('=' * 90)
for i in range(len(comparison['Method'])):
    print(f"\n{comparison['Method'][i]} ({comparison['Type'][i]})")
    print(f"  Target: {comparison['Target'][i]}")
    print(f"  Approach: {comparison['Approach'][i]}")
    print(f"  Speed: {comparison['Speed'][i]}")
    print(f"  Small Object Detection: {comparison['Small Objects'][i]}")